In [ ]:
import torch
import numpy as np
import gymnasium as gym

import torch
import torch.nn as nn
from torch.optim import Adam
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from PIL import Image
from pyvirtualdisplay import Display

# os.environ["DISPLAY"] = "99"
rand_seed=123
device = torch.device(f"cuda:{2}" if torch.cuda.is_available() else "cpu")

In [2]:
disp = Display(visible=0, size=(480, 480))
disp.start()

In [ ]:
env_name = "Reacher-v5"
env = gym.make(env_name, render_mode="rgb_array")

num_prev = 3
data_length = 1000000
obs, _ = env.reset(seed=rand_seed)

dataset = []  # List of (obs, action, image)
next_imgs = []
obs_arr = []
acts_arr = []

fail_counter = 0
for obs_counter in tqdm(range(data_length)):
    # if obs_counter + 3 >= data_length:
    #     break
    if len(next_imgs) == 0:
        rendered_img = env.render()
        # rendered_img = rendered_img[:, 50:-50, :] # Acrobot
        rendered_img = rendered_img[100:-50, 100:-100, :]
        img_resize = Image.fromarray(rendered_img).resize((64, 64)) #.resize((cropped.shape[1] // 8 * 8, cropped.shape[0] // 8 * 8))
        next_imgs.append(np.asarray(img_resize))
        obs_arr.append(obs.copy())

    act = env.action_space.sample()
    next_obs, rew, done, _, _ = env.step(act)
    acts_arr.append(act)
    
    if done:
        fail_counter += 1
        obs, _ = env.reset()
        next_imgs = []
        obs_arr = []
        acts_arr = []
    else:
        if len(next_imgs) == num_prev + 1:
            obs_arr.pop(0)
            next_imgs.pop(0)
            acts_arr.pop(0)
        next_img = env.render()
        # next_img = next_img[:, 50:-50, :] # [y dir, x dir, color]
        next_img = next_img[100:-50, 100:-100, :] # [y dir, x dir, color]
        next_img = Image.fromarray(next_img).resize((64, 64)) #.resize((cropped.shape[1] // 8 * 8, cropped.shape[0] // 8 * 8)).resize((cropped.shape[1] // 8 * 8, cropped.shape[0] // 8 * 8))
        next_imgs.append(np.asarray(next_img))
        obs_arr.append(next_obs.copy())
        
        if len(next_imgs) == num_prev + 1:
            """
            fig, ax = plt.subplots(1, 3, figsize=(8, 10))
            ax[0].set_title("Current Image")
            ax[1].set_title("Current Image 2")
            ax[2].set_title("Future Image")
            for i, img in enumerate(next_imgs):
                ax[i].imshow(img)
                ax[i].set_xlabel(np.array2string(obs_arr[i], formatter={'float_kind': lambda x: "%.2f" % x}))
            plt.show()
            """
            # Store only valid transitions
            dataset.append({
                'obs': np.asarray(obs_arr).copy(),
                'action': np.asarray(acts_arr).copy(),
                'image': np.asarray(next_imgs[:-1]).copy(),
                'next_image': next_imgs[-1].copy()
            })
        obs = next_obs
    
    if (obs_counter+1) % 25000 == 0 or (obs_counter+1) == data_length:
        print("Saving at", obs_counter+1)
        all_obs = np.array([d['obs'] for d in dataset])
        all_obs = all_obs.reshape(all_obs.shape[0], -1)
        all_acts = np.array([d['action'] for d in dataset])
        all_acts = all_acts.reshape(-1, all_acts.shape[-1])
        if not os.path.exists("../data/" + env_name.lower() + "/actions.txt"):
            np.savetxt("../data/" + env_name.lower() + "/obs.txt", all_obs)
            np.savetxt("../data/" + env_name.lower() + "/actions.txt", all_acts)
        else:
            prev_obs = np.loadtxt("../data/" + env_name.lower() + "/obs.txt")
            prev_acts = np.loadtxt("../data/" + env_name.lower() + "/actions.txt")
            all_obs = np.concatenate((prev_obs, all_obs), axis=0)
            all_acts = np.concatenate((prev_acts, all_acts), axis=0)
            np.savetxt("../data/" + env_name.lower() + "/obs.txt", all_obs)
            np.savetxt("../data/" + env_name.lower() + "/actions.txt", all_acts)

        # make sure to delete old images from directory before saving new ones
        for i, d in enumerate(dataset):
            index = i + (obs_counter+1) - 25000
            for j, img in enumerate(d['image']):
                Image.fromarray(img).save(f"../data/{env_name.lower()}/{index:05d}_{j:05d}_curr.png")
            Image.fromarray(d['next_image']).save(f"../data/{env_name.lower()}/{index:05d}_next.png")
        
        next_imgs = []
        obs_arr = []
        acts_arr = []
        dataset = []

print(f"Failures (resets): {fail_counter}")

/home/jarmibe7/E2C/venv/lib/python3.8/site-packages/glfw/__init__.py:917: GLFWError: (65550) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
/home/jarmibe7/E2C/venv/lib/python3.8/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


FatalError: an OpenGL platform library has not been loaded into this process, this most likely means that a valid OpenGL context has not been created before mjr_makeContext was called

In [3]:
all_obs = np.array([d['obs'] for d in dataset])
all_obs = all_obs.reshape(all_obs.shape[0], -1)
# all_obs.reshape(all_obs.shape[0], num_prev+1, env.observation_space.shape[0]).shape
np.savetxt("../data/" + env_name.lower() + "/obs.txt", all_obs)
all_acts = np.array([d['action'] for d in dataset])
all_acts = all_acts.reshape(-1, all_acts.shape[-1])
np.savetxt("../data/" + env_name.lower() + "/actions.txt", all_acts)

# make sure to delete old images from directory before saving new ones
for i, d in tqdm(enumerate(dataset), total=len(dataset)):
    for j, img in enumerate(d['image']):
        Image.fromarray(img).save(f"../data/{env_name.lower()}/{i:05d}_{j:05d}_curr.png")
    Image.fromarray(d['next_image']).save(f"../data/{env_name.lower()}/{i:05d}_next.png")

disp.stop()

100%|██████████| 99998/99998 [01:16<00:00, 1301.14it/s]


In [ ]:
env_name = "HalfCheetah-v5"
env = gym.make(env_name, render_mode="rgb_array")

data_length = 20000
obs, _ = env.reset(seed=rand_seed)

dataset = []  # List of (obs, action, image)
next_imgs = []
obs_arr = []

num_prev = 4

fail_counter = 0
for obs_counter in tqdm(range(data_length)):
    # if obs_counter + 3 >= data_length:
    #     break
    if len(next_imgs) == 0:
        rendered_img = env.render()
        img_resize = Image.fromarray(rendered_img).resize((64, 64)) #.resize((cropped.shape[1] // 8 * 8, cropped.shape[0] // 8 * 8))
        next_imgs.append(np.asarray(img_resize))
        obs_arr.append(obs.copy())

    act = env.action_space.sample()
    next_obs, rew, done, _, _ = env.step(act)
    
    if done or obs_counter % 25 == 0:
        fail_counter += 1
        obs, _ = env.reset()
        next_imgs = []
        obs_arr = []
    else:
        if len(next_imgs) == num_prev + 1:
            obs_arr.pop(0)
            next_imgs.pop(0)
        next_img = env.render()
        next_img = Image.fromarray(next_img).resize((64, 64)) #.resize((cropped.shape[1] // 8 * 8, cropped.shape[0] // 8 * 8)).resize((cropped.shape[1] // 8 * 8, cropped.shape[0] // 8 * 8))
        next_imgs.append(np.asarray(next_img))
        obs_arr.append(next_obs.copy())
        
        if len(next_imgs) == num_prev + 1:
            """fig, ax = plt.subplots(1, 3, figsize=(8, 10))
            ax[0].set_title("Current Image")
            ax[1].set_title("Current Image 2")
            ax[2].set_title("Future Image")
            for i, img in enumerate(next_imgs):
                ax[i].imshow(img)
                ax[i].set_xlabel(np.array2string(obs_arr[i], formatter={'float_kind': lambda x: "%.2f" % x}))
            plt.show()"""
            # Store only valid transitions
            dataset.append({
                'obs': np.asarray(obs_arr).copy(),
                'action': act,
                'image': np.asarray(next_imgs[:-1]).copy(),
                'next_image': next_imgs[-1].copy()
            })
        obs = next_obs

print(f"Failures (resets): {fail_counter}")

 12%|█▏        | 2393/20000 [00:15<01:52, 156.70it/s]

In [ ]:
all_obs = np.array([d['obs'] for d in dataset])
print(all_obs.shape)
all_obs = all_obs.reshape(all_obs.shape[0]-1, -1)
# all_obs.reshape(data_length-1, 3, env.observation_space.shape[0]).shape
np.savetxt("../data/" + env_name.lower() + "/obs.txt", all_obs)
all_acts = np.array([d['action'] for d in dataset])
np.savetxt("../data/" + env_name.lower() + "/actions.txt", all_acts)

# make sure to delete old images from directory before saving new ones
for i, d in tqdm(enumerate(dataset), total=len(dataset)):
    for j, img in enumerate(d['image']):
        Image.fromarray(img).save(f"../data/{env_name.lower()}/{i:05d}_{j:05d}_curr.png")
    Image.fromarray(d['next_image']).save(f"../data/{env_name.lower()}/{i:05d}_next.png")

(19997, 5, 17)


ValueError: cannot reshape array of size 1699745 into shape (19999,newaxis)